<a href="https://colab.research.google.com/github/LorenzoJoquino/AP157/blob/main/App_Physics_157_Monte_Carlo_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Improving the Previous Ising Model 

#Function for calculating energy of lattice system
def energy_system(spin_lattice, rows_spinLat, col_spinLat):
  #first setting variables
  sum_rowInter=0
  sum_columnInter=0
  #will next add the row-row interactions
  for i in range(rows_spinLat-1):
    prod_adjrow_spins=spin_lattice[i]*spin_lattice[i+1]
    sum_rowInter+=sum(prod_adjrow_spins)
  #will now transpose lattice in order to add the column-column product interactions
  spin_lattice_transpose=spin_lattice.transpose()
  #will add the column-column interactions by adding the row-row interactions of the transpose
  for j in range(col_spinLat-1):
    prod_adjcol_spins=spin_lattice_transpose[j]*spin_lattice_transpose[j+1]
    sum_columnInter+=sum(prod_adjcol_spins)
  sum_Inter=sum_rowInter+sum_columnInter
  energy = sum_Inter*(-J)
  return energy
#-----------------


print("The initial energy of the system is " + str(energy_system(initial_spins, rows, columns)))
print("The initial magnetization of the system is " + str(magnetization_system(initial_spins)))

Archived

In [ ]:
import numpy as np
import random as rnd
import matplotlib.pyplot as plt

Initialize Variables 

In [ ]:
J=1
k=1  #boltzmann constant
T=100  #temperature
beta=1/(k*T) #thermodynamic constant
B = 0

##for style, will ask these variables as inputs 
#steps=int(input("How mamy steps do you want it to process? "))
steps=60000
steps_x_axis=[x for x in range(0,steps+1)]

#P_spin_0=float(input("How do you want your initial probability for spin-up? "))
P_spin_0=0

rows=10
columns=10

Initialize spin system (matrix)

In [ ]:
#Block of code assigning initial spins randomly
def initialize_spin(rows, columns):
  spin=[]
  for i in range(rows*columns):
    #code that does the random picking (like coinflipping))
    assign_spin= 2*np.random.binomial(1, P_spin_0) - 1  #this resolves spin to -1 or 1
    spin.append(assign_spin)
  #must next turn it into np-type for resphaing 
  spin_list=np.array(spin)

  initial_spins=spin_list.reshape(rows,columns)  #keep this variable in mind
  return initial_spins

#next, printing output here for testing
#print(initial_spins)

In [ ]:
#Function for calculating magnetization of lattice system 
def magnetization_system(spin_lattice):
  magnetization = sum(sum(spin_lattice))
  #when one sum operator is used, it will add all elements along the column... 
  #...thus, must sum it again to turn it into a single number
  return magnetization
#----------------------

Old way of computing energy: but we must optimize it 
1. Allow multiple kinds of neighbors considered  
2. Including magnetic field into the hamiltonian 
3. Using energy difference instead of getting energies before and after and then comparing them only after 

In [ ]:
def compute_changeinE_VN(spin_lattice, x, y): 
  sum_neighbors = spin_lattice[(x+1)%rows][y] + spin_lattice[x-1][y] + spin_lattice[x][(y+1)%columns] + spin_lattice[x][y-1]
  #need to adjust for boundary problems first 


  #change_inE = (-J*sum_neighbors - B)*-2*spin_lattice[x][y]
  change_inE = (J*sum_neighbors + B)*2*spin_lattice[x][y] #instead of flipped spin - current spin, I just multiplied current spin by -2 to show difference 
  return change_inE

#compute_changeinE_VN(initial_spins, 5, 5) #just for checking 

In [ ]:
#Let us first define a function for the acceptance probability 
def Accept_or_Reject(E_change):
  if E_change <= 0:
    return -1 
  if E_change > 0:
    accept_posEchange=  (np.e)**(-beta*(E_change))
    accept_multiplier= -2*np.random.binomial(1,accept_posEchange) + 1
    return accept_multiplier

In [ ]:
#New code to run simulation (Metropolis-style) March 2, 2022


In [ ]:
#Now, let us run the Monte-Carlo simulation (Metropolis-style)

#must first set some more variables (y-axis relevant variables)
before_spins=initial_spins
magnetization_y_axis = [magnetization_system(initial_spins)]
energy_y_axis = [energy_system(initial_spins, rows, columns)]

 #ACTUAL LOOP for SIMULATION Code
for q in range(steps):
  #choosing a random site
  site_rowChosen=rnd.randint(0,rows-1)
  site_colChosen=rnd.randint(0,columns-1)

  #calculating energy before flip at the start...
  #... if q>0, the energy_before would have been substituted already ...
  #...with the more recent one (see last line)
  if q==0:
    energy_before= energy_system(before_spins, rows, columns)
  
  #will now flip the spin of chosen site (as a proposal)
  after_spins=before_spins
  after_spins[site_rowChosen][site_colChosen]=-after_spins[site_rowChosen][site_colChosen]
  
  #calculating energy after flip
  energy_after=energy_system(after_spins, rows, columns)
  
  #verifying acceptance...
  test_holder=Accept_or_Reject(energy_after, energy_before, after_spins[site_rowChosen][site_colChosen])
  
  #We need to revert the energy_after to energy_before if the flip was rejected
  if test_holder!=after_spins[site_rowChosen][site_colChosen]:
    energy_after=energy_before
  #we add this line as well just in case a rejection happens (this will have no effect if flip is accepted)
  after_spins[site_rowChosen][site_colChosen]=test_holder
  
  #calculating new magnetization and adding it to the list to be used in plotting
  new_mag=magnetization_system(after_spins)
  magnetization_y_axis.append(new_mag)

  #will append the energy level as well in order to plot the energy as well 
  energy_y_axis.append(energy_after)

  #The new set of spins should become the old set for the next iteration
  before_spins=after_spins

  #will substitute the "new" energy as the "old" energy
  energy_before=energy_after


#Now Plotting the results (Magnetization vs step#)

plt.plot(steps_x_axis, magnetization_y_axis)
plt.xlabel('steps')
plt.ylabel('magnetization')
plt.title('MCMC simulation Lattice System')
plt.show()

#Now Plotting energy vs steps also
plt.plot(steps_x_axis, energy_y_axis)
plt.ylabel('energy')
plt.xlabel('steps')
plt.title('Energy of Lattice System')
plt.show()

## In case we like to show final configuration
#print("Final-state spins: ")
#print(before_spins)

print("Final Stats:")
print("The final energy of the system is " + str(energy_after))
print("The final magnetization is " + str(new_mag))